### Connexion

In [1]:
import os
from dotenv import load_dotenv
import pyodbc

# Charger les variables d'environnement
load_dotenv()

server = os.getenv('DB_SERVER')
database = os.getenv('DB_DATABASE')
username = os.getenv('DB_USERNAME')
password = os.getenv('DB_PASSWORD')

driver= '{ODBC Driver 17 for SQL Server}'

cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

### nettoyage partie 1 les URL de redirection ( refaire le code pour qu'il ne netoie seulement seux qui on la date la plus elever )

In [3]:
# import requests
# import pyodbc

# # Connexion à la base de données
# cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
# cursor = cnxn.cursor()

# # Sélection des URL_Produit de la table Produit
# cursor.execute("SELECT ID_Produit, URL_Produit FROM Produit")
# rows = cursor.fetchall()

# for row in rows:
#     id_produit = row[0]
#     url_produit = row[1]
    
#     try:
#         # Suivre les redirections pour obtenir l'URL finale
#         response = requests.get(url_produit)
#         url_final = response.url
        
#         # Mettre à jour l'URL_Produit dans la table Produit si une redirection a eu lieu
#         if url_produit != url_final:
#             cursor.execute("UPDATE Produit SET URL_Produit = ? WHERE ID_Produit = ?", url_final, id_produit)
#     except requests.exceptions.RequestException as e:
#         print(f"Une erreur s'est produite lors de la tentative d'accès à l'URL {url_produit} : {e}")

# cnxn.commit()
# cnxn.close()


Une erreur s'est produite lors de la tentative d'accès à l'URL https://www.carter-cash.com/pneus/sur-commande/225-50r18-99h-continental-hiver-1013018104 : ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


In [2]:
import pyodbc
import requests

# Connexion à la base de données
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

# Sélection de la date la plus élevée dans la colonne Date_scrap de la table Produit
cursor.execute("SELECT MAX(Date_scrap) FROM Produit")
max_date = cursor.fetchone()[0]

# Sélection des tables qui ont la date la plus élevée dans la colonne Date_scrap de la table Produit
cursor.execute("SELECT ID_Produit, URL_Produit FROM Produit WHERE Date_scrap = ?", max_date)
rows = cursor.fetchall()

for row in rows:
    id_produit = row[0]
    url_produit = row[1]
    
    try:
        # Suivre les redirections pour obtenir l'URL finale
        response = requests.get(url_produit)
        url_final = response.url
        
        # Mettre à jour l'URL_Produit dans la table Produit si une redirection a eu lieu
        if url_produit != url_final:
            cursor.execute("UPDATE Produit SET URL_Produit = ? WHERE ID_Produit = ?", url_final, id_produit)
    except requests.exceptions.RequestException as e:
        print(f"Une erreur s'est produite lors de la tentative d'accès à l'URL {url_produit} : {e}")

cnxn.commit()
cnxn.close()


### nettoyage partie 2 les doublons 

In [4]:
import pyodbc

# Connexion à la base de données
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

# Suppression des doublons dans la table Produit
cursor.execute("""
WITH CTE AS(
   SELECT ID_Produit, URL_Produit, Date_scrap,
       RN = ROW_NUMBER()OVER(PARTITION BY URL_Produit, Prix, Date_scrap ORDER BY ID_Produit)
   FROM Produit
)
SELECT ID_Produit FROM CTE WHERE RN > 1
""")
duplicates = cursor.fetchall()

for id_produit in duplicates:
    # Suppression des doublons dans les autres tables
    for table_name in ['Caracteristiques', 'Dimensions']:
        cursor.execute(f"DELETE FROM {table_name} WHERE ID_Produit = ?", id_produit)

    # Suppression des doublons dans la table Produit
    cursor.execute("DELETE FROM Produit WHERE ID_Produit = ?", id_produit)

cnxn.commit()
cnxn.close()


### nettoyage partie 3 les prix par rapport à l'url

In [16]:
# from lxml import html
# import requests
# import pyodbc

# # Connexion à la base de données
# cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
# cursor = cnxn.cursor()

# # Sélection des URL_Produit, Prix et Date_scrap de la table Produit
# cursor.execute("SELECT ID_Produit, URL_Produit, Prix, Date_scrap FROM Produit")
# rows = cursor.fetchall()

# # Trouver la date la plus récente
# date_max = max(row[3] for row in rows)

# for row in rows:
#     id_produit = row[0]
#     url_produit = row[1]
#     prix_produit = row[2]
#     date_scrap = row[3]
    
#     # Vérifier si la date de scraping est la plus récente
#     if date_scrap == date_max:
#         # Obtenir le contenu HTML de la page
#         response = requests.get(url_produit)
#         tree = html.fromstring(response.content)
        
#         # Extraire le prix de la page
#         prix_page_list = tree.xpath('//*[@id="tire"]/div[2]/div[3]/div/div[3]/div[2]/div[2]/div[1]/form/div[1]/div[1]/div/div/span/text()')
        
#         # Vérifier si la liste est vide
#         if prix_page_list:
#             prix_page = prix_page_list[0]
#         else:
#             prix_page = 666
        
#         # Comparer le prix de la page avec le prix dans la base de données
#         if prix_produit != prix_page:
#             # Mettre à jour le prix dans la base de données
#             cursor.execute("UPDATE Produit SET Prix = ? WHERE ID_Produit = ?", prix_page, id_produit)

# cnxn.commit()
# cnxn.close()


In [5]:
from lxml import html
import requests
import pyodbc

# Connexion à la base de données
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

# Sélection de la date la plus élevée dans la colonne Date_scrap de la table Produit
cursor.execute("SELECT MAX(Date_scrap) FROM Produit")
date_max = cursor.fetchone()[0]

# Sélection des URL_Produit, Prix et Date_scrap de la table Produit pour la date la plus élevée
cursor.execute("SELECT ID_Produit, URL_Produit, Prix, Date_scrap FROM Produit WHERE Date_scrap = ?", date_max)
rows = cursor.fetchall()

for row in rows:
    id_produit = row[0]
    url_produit = row[1]
    prix_produit = row[2]
    
    # Obtenir le contenu HTML de la page
    response = requests.get(url_produit)
    tree = html.fromstring(response.content)
    
    # Extraire le prix de la page
    prix_page_list = tree.xpath('//*[@id="tire"]/div[2]/div[3]/div/div[3]/div[2]/div[2]/div[1]/form/div[1]/div[1]/div/div/span/text()')
    
    # Vérifier si la liste est vide
    if prix_page_list:
        prix_page = prix_page_list[0]
    else:
        prix_page = 666
    
    # Comparer le prix de la page avec le prix dans la base de données
    if prix_produit != prix_page:
        # Mettre à jour le prix dans la base de données
        cursor.execute("UPDATE Produit SET Prix = ? WHERE ID_Produit = ?", prix_page, id_produit)

cnxn.commit()
cnxn.close()


<span style="color:orange"> récupérer le premier mot de la colonne ‘descriptif’ seulement quand la colonne ‘Marque’ est vide, </span>

In [6]:
import pyodbc

# Connexion à la base de données
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

# Mise à jour de la colonne Marque dans la table Produit
cursor.execute("""
UPDATE Produit
SET Marque = CASE 
    WHEN CHARINDEX(' ', Descriptif) > 0 THEN LEFT(Descriptif, CHARINDEX(' ', Descriptif) - 1)
    ELSE Descriptif
END
WHERE Marque IS NULL OR Marque = ''
""")

cnxn.commit()
cnxn.close()

print("La colonne 'Marque' a été mise à jour avec succès.")



La colonne 'Marque' a été mise à jour avec succès.
